### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [2]:
import os
from dotenv import load_dotenv

from langchain_groq import ChatGroq


In [3]:
load_dotenv()

# grop_api_key = os.getenv("GROQ_API_KEY")
# grop_api_key

'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'

## Model Downlaod


In [4]:
model = ChatGroq(model='gemma2-9b-it', api_key=grop_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10b609360>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10b6086a0>, model_name='gemma2-9b-it', groq_api_key=SecretStr('**********'))

In [5]:
model.invoke("what are job profiles for LLm in data science")

AIMessage(content='LLMs (Large Language Models) are revolutionizing the field of data science, offering powerful capabilities for processing and understanding natural language. \n\nHere are some job profiles where LLMs are making a significant impact:\n\n**1. NLP Engineer/Scientist:**\n\n* **Focus:** Develop and implement NLP solutions using LLMs.\n* **Tasks:**\n    * Fine-tune LLMs for specific data science tasks like text classification, sentiment analysis, question answering, and summarization.\n    * Build and maintain NLP pipelines using LLMs as core components.\n    * Design and evaluate new NLP algorithms and models leveraging LLMs.\n    * Collaborate with data scientists and other engineers to integrate LLMs into data science workflows.\n\n**2. Data Analyst/Scientist with NLP Specialization:**\n\n* **Focus:** Analyze and interpret textual data using LLMs to uncover insights and patterns.\n* **Tasks:**\n    * Extract meaningful information from unstructured text data using LLMs 

In [7]:
from langchain_core.messages import HumanMessage, AIMessage , SystemMessage


prompt_template = [
        SystemMessage(content="you are expert real state agent and proffessional in this"),
        HumanMessage(content="how to calculate builtup are when efficiency and carpet is give provide formula")
]

prompt_template

[SystemMessage(content='you are expert real state agent and proffessional in this'),
 HumanMessage(content='how to calculate builtup are when efficiency and carpet is give provide formula')]

In [9]:
response = model.invoke(prompt_template)

In [11]:
print(response.content)

You're asking the right questions!  Calculating built-up area accurately is crucial in real estate. Here's a breakdown of how to do it when you're given efficiency and carpet area:

**Understanding the Terms**

* **Carpet Area:** This is the actual usable floor space within the walls of a unit. It's the area you'd measure with a tape measure, excluding balconies, walls, and other structural elements.
* **Built-up Area:** This includes the carpet area *plus* the area of the walls, balconies, and any other enclosed spaces. 

* **Efficiency:** This is the percentage of built-up area that constitutes the usable carpet area. It essentially tells you how much of the total space is actually living space.

**Formula**

Here's the formula to calculate built-up area:

**Built-up Area = Carpet Area / (Efficiency / 100)**

**Example**

Let's say:

* **Carpet Area:** 1000 square feet
* **Efficiency:** 70%

**Calculation:**

Built-up Area = 1000 square feet / (70 / 100)
Built-up Area = 1000 square f

In [12]:
from langchain_core.output_parsers import StrOutputParser

output = StrOutputParser()

output.invoke(response)

"You're asking the right questions!  Calculating built-up area accurately is crucial in real estate. Here's a breakdown of how to do it when you're given efficiency and carpet area:\n\n**Understanding the Terms**\n\n* **Carpet Area:** This is the actual usable floor space within the walls of a unit. It's the area you'd measure with a tape measure, excluding balconies, walls, and other structural elements.\n* **Built-up Area:** This includes the carpet area *plus* the area of the walls, balconies, and any other enclosed spaces. \n\n* **Efficiency:** This is the percentage of built-up area that constitutes the usable carpet area. It essentially tells you how much of the total space is actually living space.\n\n**Formula**\n\nHere's the formula to calculate built-up area:\n\n**Built-up Area = Carpet Area / (Efficiency / 100)**\n\n**Example**\n\nLet's say:\n\n* **Carpet Area:** 1000 square feet\n* **Efficiency:** 70%\n\n**Calculation:**\n\nBuilt-up Area = 1000 square feet / (70 / 100)\nBui

In [14]:
chain = model|output

In [15]:
chain.invoke(prompt_template)

"You're asking about calculating built-up area when you're given the efficiency and carpet area. Here's the breakdown:\n\n**Understanding the Terms**\n\n* **Carpet Area:** This is the usable floor area within the walls of a property, excluding balconies, external walls, and internal walls. \n* **Built-up Area:** This is the total covered area of a property, including the carpet area, balconies, external walls, and internal walls.\n* **Efficiency:** This is the ratio of the built-up area to the carpet area. It's expressed as a percentage.\n\n**Formula**\n\n**Built-up Area = Carpet Area * (100 / Efficiency)**\n\n**Example**\n\nLet's say you have a property with:\n\n* Carpet Area: 1000 square feet\n* Efficiency: 80%\n\n**Calculation:**\n\nBuilt-up Area = 1000 square feet * (100 / 80) = 1250 square feet\n\n**Important Considerations**\n\n* **Local Standards:**  Building codes and regulations can vary by location.  Make sure you're using the correct definition of built-up area and efficienc

## New Prompt template

In [18]:
### Prompt Templates

from langchain_core.prompts import ChatPromptTemplate

generic_prompt = "translate the following text into {language}"

prompt = ChatPromptTemplate.from_messages (
    [("system" , generic_prompt),("user","{text}")]
)

In [22]:
result = prompt.invoke({"language":"French","text":"Hello my name is Sachin"})
result

ChatPromptValue(messages=[SystemMessage(content='translate the following text into French'), HumanMessage(content='Hello my name is Sachin')])

In [28]:
result.to_messages()

[SystemMessage(content='translate the following text into French'),
 HumanMessage(content='Hello my name is Sachin')]

In [20]:
chain = prompt|model|output

In [21]:
chain.invoke({"language":"French","text":"Hello my name is Sachin"})

"Bonjour, je m'appelle Sachin. \n"

In [31]:

generic_prompt1 = "translate the following text into {language} based on {text}"
prompt2 =ChatPromptTemplate.from_template(generic_prompt1)
prompt2

ChatPromptTemplate(input_variables=['language', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], template='translate the following text into {language} based on {text}'))])

In [34]:
result = prompt2.invoke({"language":"French","text":"Hello my name is Sachin"})
result.to_messages()


[HumanMessage(content='translate the following text into French based on Hello my name is Sachin')]

In [35]:
chain2 = prompt2|model|output

In [37]:
chain2.invoke({"language":"French","text":"Hello my name is Sachin"})

"Bonjour, je m'appelle Sachin. \n"